In [1]:
import numpy as np
from ondelette import wavelets
import pywt
import tensorflow as tf

np.set_printoptions(floatmode='unique')
rng = np.random.default_rng()

2025-02-20 19:39:23.795743: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [61]:
dtype = tf.float32
wavelet_name = 'db2'

wavelet = wavelets.all[wavelet_name](dtype=dtype)

samples_np = rng.integers(-32768, 32767, [2, 12], dtype=np.int16, endpoint=True)
print('SAMPLES:')
print(samples_np)
print()
samples = tf.constant(samples_np, dtype=dtype)

print(f'SAMPLE SHAPE: {samples.shape}')

a0, d0 = wavelets.dwt(wavelet, samples)
print(f'a0 SHAPE: {a0.shape}')

a1, d1 = wavelets.dwt(wavelet, a0)
print(f'a1 SHAPE: {a1.shape}')

a2, d2 = wavelets.dwt(wavelet, a1)
print(f'a2 SHAPE: {a2.shape}')

d1_pad = ((d2.shape[1] * 2) - d1.shape[1]) // 2
d1 = tf.pad(d1, [[0, 0], [d1_pad, d1_pad]])

d0_pad = (((d1.shape[1] * 2) - d0.shape[1])) // 2
d0 = tf.pad(d0, [[0, 0], [d0_pad, d0_pad]])

# r0 = wavelets.idwt(wavelet, a1, d1)
# rec_float = wavelets.idwt(wavelet, r0, d0)
# rec_rounded = tf.round(rec_float)
# rec = tf.cast(rec_rounded, tf.int16)
# print('RECONSTRUCTION:')
# print(rec.numpy())
# print()

SAMPLES:
[[  7056 -20201 -15802  -6322 -30602 -28170  13987  25370  -6581  30526
   32752  14488]
 [  -791   2240  -4355   6919   7325  17282 -23582  14476   1783  16446
  -24585  -9516]]

SAMPLE SHAPE: (2, 12)
a0 SHAPE: (2, 7)
a1 SHAPE: (2, 4)
a2 SHAPE: (2, 3)


In [62]:
print(f'APPROXIMATION: {a2.shape}')
print(a2.numpy())
print()
print(f'DETAIL 2: {d2.shape}')
print(d2.numpy())
print()
print(f'DETAIL 1: {d1.shape}')
print(d1.numpy())
print()
print(f'DETAIL 0: {d0.shape}')
print(d0.numpy())

APPROXIMATION: (2, 3)
[[  1997.5181  -20911.459    15207.108  ]
 [   149.68118    459.70367   7451.672  ]]

DETAIL 2: (2, 3)
[[  7454.839   -49875.74     -4074.732  ]
 [   558.61774   6015.268    -1996.6696 ]]

DETAIL 1: (2, 6)
[[     0.       11340.8125    8185.393    16143.309    14215.223
       0.     ]
 [     0.        -207.39052  -2502.6692   -8665.635   -18778.408
       0.     ]]

DETAIL 0: (2, 11)
[[     0.          0.      15658.793   -6550.523   -8532.04     9721.909
  -27744.621   14409.845   -7485.8174      0.          0.    ]
 [     0.          0.      -1743.5212  -7384.368   -3206.356  -31539.678
   -6644.2705 -19886.887    5314.4863      0.          0.    ]]


In [63]:
a2 = tf.pad(a2, [[0, 0], [1, 1]])
d2 = tf.pad(d2, [[0, 0], [1, 1]])
wavelets.idwt(wavelet, a2, d2)

<tf.Tensor: shape=(2, 8), dtype=float32, numpy=
array([[-2.1217769e-05,  4.1219755e-05,  3.0387622e+03, -1.0172345e+04,
        -3.8537266e+04,  4.0428594e+04,  3.1317430e-04, -2.0278923e-04],
       [ 4.4786448e-06,  1.7078193e-05, -5.5570171e+01, -1.2528976e+03,
         8.9921787e+03,  3.7163450e+03, -2.1040768e-04,  9.8679520e-06]],
      dtype=float32)>